In [2]:
# Import dependencies
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import pickle

In [3]:
# Read data
loan_train_df = pd.read_csv(r"Dataset/loan_sanction_train.csv")

In [4]:
# Clean dataset and convert categorical to numeric
loan_train_clean_df = loan_train_df.drop(columns=["Loan_ID"])
loan_train_clean_df = pd.get_dummies(loan_train_clean_df, columns=["Gender",
                                                                   "Married",
                                                                   "Dependents",
                                                                   "Education",
                                                                   "Self_Employed",
                                                                   "Property_Area"])
loan_train_clean_df = loan_train_clean_df.dropna()


In [5]:
# label data
X = loan_train_clean_df.drop(columns=["Loan_Status"]).values
y = loan_train_clean_df["Loan_Status"].values


In [6]:
# Encode the target variable (Loan_Status) to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [7]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

X_scaler = StandardScaler()
X_scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Create the deep learning model
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  64
hidden_nodes_layer2 = 32
hidden_nodes_layer3 = 1


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1344      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3457 (13.50 KB)
Trainable params: 3457 (13.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
11/11 [==============================] - 2s 57ms/step - loss: 0.8414 - accuracy: 0.3799 - val_loss: 0.7193 - val_accuracy: 0.4819
Epoch 2/50
11/11 [==============================] - 0s 10ms/step - loss: 0.6758 - accuracy: 0.5775 - val_loss: 0.6449 - val_accuracy: 0.6627
Epoch 3/50
11/11 [==============================] - 0s 10ms/step - loss: 0.6088 - accuracy: 0.7112 - val_loss: 0.6145 - val_accuracy: 0.6867
Epoch 4/50
11/11 [==============================] - 0s 17ms/step - loss: 0.5693 - accuracy: 0.7447 - val_loss: 0.5933 - val_accuracy: 0.7229
Epoch 5/50
11/11 [==============================] - 0s 11ms/step - loss: 0.5358 - accuracy: 0.7720 - val_loss: 0.5714 - val_accuracy: 0.7590
Epoch 6/50
11/11 [==============================] - 0s 10ms/step - loss: 0.5038 - accuracy: 0.7903 - val_loss: 0.5540 - val_accuracy: 0.7952
Epoch 7/50
11/11 [==============================] - 0s 10ms/step - loss: 0.4765 - accuracy: 0.8207 - val_loss: 0.5365 - val_accuracy: 0.7952
Epoch 8/50
11

In [10]:
# Evaluate the model on test data
loss, accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")

5/5 - 0s - loss: 0.6813 - accuracy: 0.7536 - 50ms/epoch - 10ms/step
Loss: 0.68131023645401, Accuracy: 0.7536231875419617


In [11]:
# Save the model to a file
filename = 'deep_learning_model.h5'
nn.save(filename)

# Load the model from the file and check its accuracy
loaded_model = tf.keras.models.load_model(filename)
loss, accuracy = loaded_model.evaluate(X_test_scaled, y_test)
print("Loaded Model Test Accuracy:", accuracy)

c:\Users\Brecht Nys\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5/5 [==============================] - 0s 3ms/step - loss: 0.6813 - accuracy: 0.7536
Loaded Model Test Accuracy: 0.7536231875419617
